In [5]:
import json
import datetime
import boto3
from boto3.dynamodb.conditions import Key, Attr

from secrets import AwsAccesKeyID
from secrets import AwsScretKey

In [4]:
#- Reference Dynamo
dynamodb = boto3.resource('dynamodb',
                        region_name='us-west-2',
                        aws_access_key_id = AwsAccesKeyID,
                        aws_secret_access_key = AwsScretKey)

#- Access Table
dynamoTable = dynamodb.Table("chat")

print("completed")

completed


In [6]:
#- Scan Table
response = dynamoTable.scan(Key('chat_id'))

In [7]:
response

{'Items': [{'chat_id': '2', 'timestamp': Decimal('1561431042000')},
  {'chat_id': '1', 'timestamp': Decimal('1561429242000')},
  {'message': 'yeah totally!',
   'shopping_list': [['apples', 'oranges'], ['Straw', 'blue berry', 'stuff']],
   'chat_id': '5',
   'timestamp': Decimal('1561417617000')},
  {'message': 'yeah totally!',
   'chat_id': '4',
   'timestamp': Decimal('1561417617000')},
  {'chat_id': '3', 'timestamp': Decimal('1561417617000')}],
 'Count': 5,
 'ScannedCount': 5,
 'ResponseMetadata': {'RequestId': '6OG6BQ0Q5PLP5EB48PANG662R7VV4KQNSO5AEMVJF66Q9ASUAAJG',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'Server',
   'date': 'Wed, 03 Jul 2019 19:47:07 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '500',
   'connection': 'keep-alive',
   'x-amzn-requestid': '6OG6BQ0Q5PLP5EB48PANG662R7VV4KQNSO5AEMVJF66Q9ASUAAJG',
   'x-amz-crc32': '77317701'},
  'RetryAttempts': 0}}

In [ ]:
#1561417617000


In [13]:
testTable = dynamodb.create_table(
    TableName="Test",
    KeySchema=[
        {
            'AttributeName' : "Date",
            'KeyType': "HASH"
        },
        {
            'AttributeName' : "Info",
            'KeyType': "RANGE"
        }
    ],
    AttributeDefinitions=[
        {
            'AttributeName' : "Date",
            'AttributeType': 'N'
        },
        {
            'AttributeName' : "Info",
            'AttributeType': 'S'
        }
    ],
    ProvisionedThroughput={
        'ReadCapacityUnits': 5,
        'WriteCapacityUnits': 5
    }
    )

print('completed')

completed


In [15]:
testTable.put_item(Item={
    "Date" : 123,
    "Info" : "kdfkdjf"
})

{'ResponseMetadata': {'RequestId': 'MMFDI28AJDEAF9O676S5H7PTSFVV4KQNSO5AEMVJF66Q9ASUAAJG',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'Server',
   'date': 'Wed, 03 Jul 2019 20:03:22 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '2',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'MMFDI28AJDEAF9O676S5H7PTSFVV4KQNSO5AEMVJF66Q9ASUAAJG',
   'x-amz-crc32': '2745614147'},
  'RetryAttempts': 0}}

In [18]:


testTable2 = dynamodb.create_table(
    TableName = "Test2",
    # Declare Primary Key with KeySchema
    KeySchema =[
        {
            "AttributeName" : "ID",
            "KeyType": "HASH"
        }
    ],
    # Declare AttributeDefinition
    AttributeDefinitions=[
        {
            "AttributeName" : "ID",
            "AttributeType" : "S"
        }
    ],
    ProvisionedThroughput={
        'ReadCapacityUnits': 5,
        'WriteCapacityUnits': 5
    }
    )

print("--> Completed")

--> Completed


In [19]:
sourceFile = "articles_all.json"
    
with open(sourceFile) as jsonFile:
    sourceData = json.load(jsonFile)
    
print("completed loading file")

completed loading file


In [32]:
#-- Create Secondary Index
dynamodbClient = boto3.client('dynamodb',
                        region_name='us-west-2',
                        aws_access_key_id = AwsAccesKeyID,
                        aws_secret_access_key = AwsScretKey)

dynamodbClient.update_table(
    TableName = "Test2",
    AttributeDefinitions =[
        {
            "AttributeName" : "publishdate",
            "AttributeType" : "N"
        }
    ],
    GlobalSecondaryIndexUpdates=[
        {
            "Create" : {
                "IndexName": "PublishDateIndex",
                "KeySchema" : [
                    {
                        "AttributeName" : "publishdate",
                        "KeyType" : "HASH"
                    }
                ],
                "Projection" : {
                    "ProjectionType" : "ALL"
                },
                "ProvisionedThroughput": {
                        "ReadCapacityUnits": 5,
                        "WriteCapacityUnits": 5,
                }
            }
        }
    ]
)

print("Completed secondary index")

Completed secondary index


In [25]:
counter = 0

for article in sourceData['articles']:
    
    sourceImageUrl = article['imageurl']
    
    if (sourceImageUrl == ""):
        sourceImageUrl = "NA"
    
    testTable2.put_item(Item={
        "ID" : article["id"],
        "title" : article['title'],
        "sourceurl" : article['sourceurl'],
        "imageurl" : sourceImageUrl,
        "publishdate" : int(article['publishdate'])
        })
    
    
    counter +=1
    if (counter == 20):
        break
        
print("completed")

completed


In [28]:
#-- Query By ID
queryResponse = testTable2.query(KeyConditionExpression=Key('ID').eq('02812443-4a7a-47c0-9f18-6f22fba18274'))

queryResponse

{'Items': [{'publishdate': Decimal('20180101'),
   'ID': '02812443-4a7a-47c0-9f18-6f22fba18274',
   'sourceurl': 'https://www.nytimes.com/2018/01/01/business/media/rupert-murdoch-21st-century-fox-disney.html',
   'imageurl': 'https://static01.nyt.com/images/2018/01/02/business/02MURDOCHEMPIRE-01/02MURDOCHEMPIRE-1-articleLarge-v2.jpg',
   'title': 'With Disney Deal Looming, Murdoch’s Empire Is Fractured'}],
 'Count': 1,
 'ScannedCount': 1,
 'ResponseMetadata': {'RequestId': '61BTDOIPE8HM0HG2KPE7EL2L2NVV4KQNSO5AEMVJF66Q9ASUAAJG',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'Server',
   'date': 'Wed, 03 Jul 2019 20:31:51 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '437',
   'connection': 'keep-alive',
   'x-amzn-requestid': '61BTDOIPE8HM0HG2KPE7EL2L2NVV4KQNSO5AEMVJF66Q9ASUAAJG',
   'x-amz-crc32': '1852646455'},
  'RetryAttempts': 0}}

In [34]:
#-- Query By Publish Date
query2Response = testTable2.query(IndexName="PublishDateIndex", 
                                  KeyConditionExpression=Key("publishdate").eq(20180101))

query2Response

{'Items': [{'publishdate': Decimal('20180101'),
   'ID': 'e59f32db-cc2d-4da4-bb07-33c80cd8650e',
   'sourceurl': 'https://www.nytimes.com/2018/01/01/business/airlines-travel-entertainment.html',
   'imageurl': 'https://static01.nyt.com/images/2018/01/02/business/02ITINERARIES1/merlin_131591759_4fc40064-d694-436f-a807-2aca524d21eb-articleLarge.jpg',
   'title': 'Those Seatback Screens on Planes Are Starting to Disappear'},
  {'publishdate': Decimal('20180101'),
   'ID': '02cbbbc0-1dbf-4476-9c1a-fa1cd829d089',
   'sourceurl': 'https://www.nytimes.com/2018/01/01/business/sydney-compass-richard-cousins.html',
   'imageurl': 'https://static01.nyt.com/images/2018/01/02/business/02compass/02compass-articleLarge.jpg',
   'title': 'Chief of Compass, U.K. Catering Giant, Is Killed in Plane Crash'},
  {'publishdate': Decimal('20180101'),
   'ID': '38f107cc-4c22-4c2a-b579-b57795dc73e0',
   'sourceurl': 'https://www.nytimes.com/2018/01/01/business/energy-environment/climate-change-enzymes-laundry.h

In [40]:
for item in query2Response['Items']:
    print(item['title'])
    print("----")

Those Seatback Screens on Planes Are Starting to Disappear
----
Chief of Compass, U.K. Catering Giant, Is Killed in Plane Crash
----
Fighting Climate Change, One Laundry Load at a Time
----
With Disney Deal Looming, Murdoch’s Empire Is Fractured
----
Copying iPhone Messages to the Computer
----
How Antivirus Software Can Be Turned Into a Tool for Spying
----
A Big Year for the Stock Market
----
Start of a New Year of Trading, and Jobs Report From December
----
